In [30]:
import os
import numpy as np
import gin
from torch.nn.functional import interpolate
from PIL import Image

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from network import network
from image_retrieval import rank_images
from network import network
from pose_prediction import predictor
from datasets import base_dataset

In [4]:
from pose_prediction.sparse_to_dense_featuremetric_predictor import SparseToDenseFeatureMetricPnP
from datasets import dataload_helpers as data_helpers

In [5]:
DATA_PATH = "/home/aunagar/Personal/Others/learning/feature detection/robotcar/"

# put triangulation file in the same folder as robotcar data
triangulation_file = DATA_PATH + 'robotcar_triangulation.npz'
nvm_filepath = DATA_PATH + '3D-models/all-merged/all.nvm'
image_root = DATA_PATH + 'images/'

In [6]:
ref_image = [image_root + 'overcast-reference/rear/1417176458999821.jpg']
query_image = [image_root + 'overcast-reference/rear/1417176511810439.jpg']

filenames = ref_image + query_image

In [7]:
filename_to_pose = data_helpers.from_nvm(nvm_filepath, filenames)
filename_to_local_reconstruction = data_helpers.load_triangulation_data(triangulation_file, filenames)
filename_to_intrinsics = data_helpers.load_intrinsics(filenames)

Found  overcast-reference/rear/1417176458999821.jpg
Found  overcast-reference/rear/1417176511810439.jpg


In [8]:
##### You can ignore this part #####
@gin.configurable
def get_dataset_loader(dataset_loader_cls):
    return dataset_loader_cls

@gin.configurable
def get_pose_predictor(pose_predictor_cls: predictor.PosePredictor,
                       dataset: base_dataset.BaseDataset,
                       network: network.ImageRetrievalModel,
                       ranks: np.ndarray,
                       log_images: bool):
    return pose_predictor_cls(dataset=dataset,
                              network=network,
                              ranks=ranks,
                              log_images=log_images)

In [9]:
# Load gin config based on dataset name
gin.parse_config_file(
    'configs/runs/run_{}_on_{}.gin'.format('sparse_to_dense', 'robotcar'))

In [10]:
net = network.ImageRetrievalModel(device = "cpu")

In [11]:
s2dPnP = SparseToDenseFeatureMetricPnP(filename_to_pose, filename_to_intrinsics, filename_to_local_reconstruction, net)

In [20]:
solved_case, query_hypercolumn, reference_hypercolumn = s2dPnP.run(query_image[0], ref_image[0])

>> Generating pose predictions using sparse-to-dense matching...
Hypercolumn size is 2048
Hypercolumn resolution is 128 x 128
Hypercolumn size is 2048
Hypercolumn resolution is 128 x 128


In [29]:
# upsampled_query_hypercolumn = interpolate(query_hypercolumn[0], size=(1024,1024),
#                                           mode = 'bilinear', align_corners=True)
upsampled_reference_hypercolumn = interpolate(reference_hypercolumn, size=(1024,1024),
                                             mode='bilinear',align_corners=True)

RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 8589934592 bytes. Error code 12 (Cannot allocate memory)


In [37]:
solved_case.reference_inliers

array([[257., 359.],
       [377., 432.],
       [212., 408.],
       [343., 412.],
       [330., 356.],
       [495., 385.],
       [560., 418.],
       [495., 481.],
       [186., 451.],
       [354., 420.],
       [192., 317.],
       [400., 384.],
       [606., 476.],
       [ 47., 259.],
       [521., 491.],
       [697., 504.],
       [332., 344.],
       [700., 534.],
       [156., 243.],
       [145., 469.],
       [239., 342.],
       [357., 477.],
       [434., 376.],
       [139., 401.],
       [485., 352.],
       [557., 498.],
       [628., 502.],
       [427., 264.],
       [660., 257.],
       [331., 471.],
       [ 19., 444.],
       [287., 371.],
       [719., 508.],
       [568., 394.],
       [ 81., 464.],
       [919., 223.],
       [288., 351.],
       [ 41., 498.],
       [573., 429.],
       [529., 389.],
       [204., 426.],
       [646., 255.],
       [141., 316.],
       [377., 377.],
       [ 64., 331.],
       [579., 237.],
       [461., 483.],
       [186.,

In [38]:
query = Image.open(query_image[0])
ref = Image.open(ref_image[0])